In [9]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd

In [3]:
'''
this function checks if a row is valid: meaning that the 
latitude, longitude, tract number, and min_lead_equantity are all valid
'''
def row_is_valid(feature_row):
    if feature_row['min_lead_quantity'] == -1:
        return False
    if feature_row['lat'] == 'unknown' or feature_row['lat'] == -1:
        return False
    if feature_row['long'] == 'unknown' or feature_row['long'] == -1:
        return False
    if feature_row['tract'] == 'unknown' or feature_row['tract'] == -1:
        return False
    return True

# loading the complete data (including water data, census data, school and lead data) from the pickle file
name_map = pickle.load(open("/Users/23amrutad/Projects/LeadProj/ProjectData/final_data.pkl", 'rb'))

# the features and labels lists for the model classification
all_features = []
labels = []

# for every valid row, adding the features (every column except for the min_lead_quantity) to all_features
# if the min_lead_quantity is greater than 5.0, add 1 to labels; otherwise, add 0
for k, v in name_map.items():
    if row_is_valid(v):
        features = []
        if v['min_lead_quantity'] >= 5.0:
            labels.append(1)
        else:
            labels.append(0)
        for f in v.keys():
            if f != 'addr' and f != 'min_lead_quantity':
                features.append(v[f])
                if type(v[f]) == type(''):
                    print(v[f])
        all_features.append(features)

In [25]:
import csv

In [27]:
distance_matrix = open('/Users/23amrutad/Projects/LeadProj/ProjectData/distance_between_schools.csv', 'r')

In [28]:
distance_matrix

<_io.TextIOWrapper name='/Users/23amrutad/Projects/LeadProj/ProjectData/distance_between_schools.csv' mode='r' encoding='UTF-8'>

In [29]:
print(type(distance_matrix))

<class '_io.TextIOWrapper'>


In [22]:
# this is the random forest model from scikit-learn

#X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)
# this is the random forest model from scikit-learn

#X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)
X = np.array(all_features)
y = np.array(labels)

results = []

for j in range(1, 20):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=j)
    maxf1 = 0
    for i in range(1, 333):
        clf = RandomForestClassifier(max_depth=None, random_state=1, n_estimators = 3*i)
        dummy_clf = DummyClassifier(strategy="uniform")
        dummy_clf.fit(X_train, y_train)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_dummy = dummy_clf.predict(X_test)
        score = f1_score(y_test, y_pred)
        score_dummy = f1_score(y_test, y_pred_dummy)
        if score > maxf1:
            maxf1 = score
        print((3*i, " -- ",  maxf1, " -- ", score, " -- ", score_dummy))
        results.append([j, 3*i, score, score_dummy])

